In [1]:
import pandas_datareader as pdr
key=" "
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.svm import SVC
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

In [2]:
data = pd.read_csv("AAPL-2.CSV")
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100600,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095352,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088353,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090540,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093165,73449600
...,...,...,...,...,...,...,...
10245,2021-08-02,146.360001,146.949997,145.250000,145.520004,145.302307,62880000
10246,2021-08-03,145.809998,148.039993,145.179993,147.360001,147.139542,64786600
10247,2021-08-04,147.270004,147.789993,146.279999,146.949997,146.730164,56368300
10248,2021-08-05,146.979996,147.839996,146.169998,147.059998,146.839996,46397700


In [3]:
df=data['Adj Close'].pct_change()*100
df=df.rename('Ret_1d')
df=df.reset_index()
df

,index,Ret_1d
0,0,NaN
1,1,-5.216700
2,2,-7.340171
3,3,2.475298
4,4,2.899271
...,...,...
10245,10245,-0.233091
10246,10246,1.264422
10247,10247,-0.278224
10248,10248,0.074853


In [4]:
for i in range(1,5):
    df['Reurn_Lag'+str(i)]=df['Ret_1d'].shift(i)
    
data1=data['Open']-data['Close']
df['Open-Close']=data1.values
df['Price']=data['Adj Close'].values   
df.head()

,index,Ret_1d,Reurn_Lag1,Reurn_Lag2,Reurn_Lag3,Reurn_Lag4,Open-Close,Price
0,0,NaN,NaN,NaN,NaN,NaN,0.000000,0.100600
1,1,-5.216700,NaN,NaN,NaN,NaN,0.000558,0.095352
2,2,-7.340171,-5.216700,NaN,NaN,NaN,0.000558,0.088353
3,3,2.475298,-7.340171,-5.216700,NaN,NaN,0.000000,0.090540
4,4,2.899271,2.475298,-7.340171,-5.2167,NaN,0.000000,0.093165


In [5]:
df['momentum_1d']=df['Price']-df['Price'].shift(periods=1)
df['momentum_2d']=df['Price']-df['Price'].shift(periods=2)
df['momentum_5d']=df['Price']-df['Price'].shift(periods=5)
df['MA_5d']=df['Price'].rolling(5).mean()
df['MA_10d']=df['Price'].rolling(10).mean()
df['MA_20d']=df['Price'].rolling(20).mean()
df['MA_50d']=df['Price'].rolling(20).mean()
df['EMA_5d']=df['Price'].ewm(5, adjust=False).mean()
df['EMA_10d']=df['Price'].ewm(10, adjust=False).mean()
df['EMA_20d']=df['Price'].ewm(20, adjust=False).mean()
df['EMA_50d']=df['Price'].ewm(50, adjust=False).mean()

df['Log returns'] = np.log(data['Close']/data['Close'].shift())
df['volatility_daily'] = df['Log returns'].std()
df['volatility_monthly'] = df['Log returns'].std()*21**.5
df['volatility_annually'] = df['Log returns'].std()*252**.5

df.dropna()

,index,Ret_1d,Reurn_Lag1,Reurn_Lag2,Reurn_Lag3,Reurn_Lag4,Open-Close,Price,momentum_1d,momentum_2d,...,MA_20d,MA_50d,EMA_5d,EMA_10d,EMA_20d,EMA_50d,Log returns,volatility_daily,volatility_monthly,volatility_annually
19,19,-0.784507,5.371803,-2.024325,-4.263295,-4.444689,0.001116,0.110660,-0.000875,0.004811,...,0.108626,0.108626,0.111756,0.109736,0.106629,0.103464,-0.007874,0.028954,0.132686,0.459638
20,20,-3.556841,-0.784507,5.371803,-2.024325,-4.263295,0.000558,0.106724,-0.003936,-0.004811,...,0.108933,0.108933,0.110918,0.109462,0.106634,0.103528,-0.036219,0.028954,0.132686,0.459638
21,21,0.409467,-3.556841,-0.784507,5.371803,-2.024325,0.000000,0.107161,0.000437,-0.003499,...,0.109523,0.109523,0.110292,0.109253,0.106659,0.103599,0.004090,0.028954,0.132686,0.459638
22,22,2.040854,0.409467,-3.556841,-0.784507,5.371803,0.000000,0.109348,0.002187,0.002624,...,0.110573,0.110573,0.110134,0.109261,0.106787,0.103712,0.020201,0.028954,0.132686,0.459638
23,23,-0.800198,2.040854,0.409467,-3.556841,-0.784507,0.000558,0.108473,-0.000875,0.001312,...,0.111469,0.111469,0.109857,0.109190,0.106867,0.103805,-0.008032,0.028954,0.132686,0.459638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10245,10245,-0.233091,0.151054,0.455235,-1.219596,-1.490040,0.839997,145.302307,-0.339478,-0.119812,...,145.592865,145.592865,145.610583,143.882172,140.018676,133.790172,-0.002334,0.028954,0.132686,0.459638
10246,10246,1.264422,-0.233091,0.151054,0.455235,-1.219596,-1.550003,147.139542,1.837235,1.497757,...,145.859465,145.859465,145.865409,144.178297,140.357765,134.051925,0.012565,0.028954,0.132686,0.459638
10247,10247,-0.278224,1.264422,-0.233091,0.151054,0.455235,0.320007,146.730164,-0.409378,1.427857,...,145.978287,145.978287,146.009535,144.410285,140.661212,134.300517,-0.002786,0.028954,0.132686,0.459638
10248,10248,0.074853,-0.278224,1.264422,-0.233091,0.151054,-0.080002,146.839996,0.109832,-0.299546,...,146.169001,146.169001,146.147945,144.631168,140.955440,134.546390,0.000748,0.028954,0.132686,0.459638
